In [6]:
import json; import requests; import datetime; import time; import datetime; import csv

In [242]:
def set_api_key():
    with open("dark_sky_api_key.txt", "r") as myfile:
        api_key = myfile.readlines()[0][:-1]

In [255]:
nyc = {'lat':'40.7128','long':'74.0060'}

In [256]:
chicago = {'lat':'41.8781','long':'87.6298'}

In [186]:
# takes a single date input in dd/mm/yy format and returns a unix timestamp
def date_string_to_unix(date_string):
    return (str(int(time.mktime(datetime.datetime.strptime(date_string, "%d/%m/%Y").timetuple()))))

In [214]:
# return a list of unixtimestamps, one for each day between d1 and d2 specified in the function body
def unix_dates_for_year(year):
    start_date = date(year, 1, 1)  # start date
    end_date = date(year, 12, 31)  # end date
    delta = end_date - start_date      # timedelta
    dates = [(start_date + datetime.timedelta(days=i)) for i in range(delta.days + 1)]
    dates = [x.strftime('%d/%m/%Y') for x in dates]
    unix_dates  = [date_string_to_unix(dates[x]) for x in range(len(dates))]
    return unix_dates

In [257]:
# takes a timestamp, a lat and long, and returns a url string for a darksky request
def url_for_day(unix_timestamp,latitude,longitude):
    base_url = 'https://api.darksky.net/forecast/'
    tail = '?exclude=currently,flags,hourly&units=si&moonPhase'
    url = "{:s}{:s}/{:s},{:s},{:s}{:s}".format(base_url,api_key,latitude,longitude,unix_timestamp,tail)
    return url

In [224]:
def set_csv_filename(year):
    return "{:s}_weather_data.txt".format(str(year))

In [250]:
def initialize_csv_headers(api_response_for_day,year):
    json_data = json.loads(api_response_for_day.text)
    keys = list(json_data['daily']['data'][0].keys())
    filename = set_csv_filename(year)
    with open(filename, 'w') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(keys)

In [251]:
def append_daily_data(api_response_for_day,year):
    json_data = json.loads(api_response_for_day.text)
    values = list(json_data['daily']['data'][0].values())
    filename = set_csv_filename(year)
    with open(filename, 'a') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(values)

In [261]:
def csv_weather_data_for_year(year,city_coord):
    lat = city_coord['lat']
    long = city_coord['long']
    unix_dates = unix_dates_for_year(year)
    initialize_csv_headers(requests.get(url_for_day(unix_dates[0],lat,long)),year)
    for x in range(10):
       append_daily_data(requests.get(url_for_day(unix_dates[x],lat,long)),year)

In [ ]:
csv_weather_data_for_year(2016,nyc)